<a href="https://colab.research.google.com/github/jobabyyy/atlas-machine_learning/blob/main/Atari_Breakout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-rl2
!pip install gym[atari]
!pip install atari-py
!pip install tensorflow==2.11.0

In [41]:
import os
import tensorflow as tf

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
!python -m atari_py.import_roms /content/drive/MyDrive/dqn/roms/

In [44]:
import gym
from gym.envs.registration import register
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

In [40]:

register(
    id='Breakout-v4',
    entry_point='gym.envs.atari:AtariEnv',
    kwargs={'game': 'breakout', 'obs_type': 'image', 'frameskip': 1},
    max_episode_steps=10000,
    nondeterministic=False,
)

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:542: UserWarning: WARN: Overriding environment Breakout-v4
  logger.warn(f"Overriding environment {spec.id}")


In [52]:
!pip install gym[accept-rom-license]


In [53]:
!pip uninstall gym
!pip install gym[atari,accept-rom-license]


Found existing installation: gym 0.25.2
Uninstalling gym-0.25.2:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/gym-0.25.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/gym/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.10/dist-packages/gym/envs/atari/__init__.py
    /usr/local/lib/python3.10/dist-packages/gym/envs/atari/environment.py
Proceed (Y/n)? y
  Successfully uninstalled gym-0.25.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 11.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.5 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827624 sha256=b7e29177ecc7dd3a9aa83cbae8f487ddb44840cfbc9eeb2ebe291583bdbe3935
  Stored in directory: /root/.cache/pip/wheels/b9/22/6d/3e7b32d98451b4cd9d12417052affbeeeea012955d437da1da
Succes

In [1]:
!pip uninstall gym
!pip install gym==0.25.2


Found existing installation: gym 0.25.2
Uninstalling gym-0.25.2:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/gym-0.25.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/gym/*
Proceed (Y/n)? y
  Successfully uninstalled gym-0.25.2
  Using cached gym-0.25.2-py3-none-any.whl


In [ ]:
import gym
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy
from gym.wrappers import AtariPreprocessing

class CompatibleAtariPreprocessing(gym.Wrapper):
    def __init__(self, env, **kwargs):
        super().__init__(env)
        self.env = AtariPreprocessing(env, **kwargs)

    def step(self, action):
        # Handling the returned values correctly
        observation, reward, done, info = self.env.step(action)
        return observation, reward, done, info

    def reset(self):
        # Add a reset method to handle environment reset
        return self.env.reset()

# Custom callback for DQNAgent
class DQNAgentCheckpoint(Callback):
    def __init__(self, filepath):
        self.filepath = filepath

    def on_epoch_end(self, epoch, logs={}):
        self.model.save_weights(self.filepath.format(epoch=epoch), overwrite=True)

# Environment setup with custom preprocessing wrapper
env = gym.make('Breakout-v4')
env = CompatibleAtariPreprocessing(env, screen_size=84, grayscale_obs=True, frame_skip=1, scale_obs=True)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

# Adjusting the model to the preprocessed observation space
input_shape = (1,) + env.observation_space.shape
model = Sequential()
model.add(Flatten(input_shape=input_shape))
model.add(Dense(24, activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(nb_actions, activation='linear'))

# Configure and compile the agent
memory = SequentialMemory(limit=50000, window_length=1)
policy = EpsGreedyQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10, target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

# Custom checkpoint to save model after every epoch
checkpoint = DQNAgentCheckpoint('policy_{epoch}.h5')

# Training
dqn.fit(env, nb_steps=80000, visualize=False, verbose=2, callbacks=[checkpoint])

# Save the final policy network after training
dqn.save_weights('policy_final.h5', overwrite=True)


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Training for 80000 steps ...


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:137: UserWarning: WARN: The obs returned by the `reset()` method was expecting a numpy array, actual type: <class 'tuple'>
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:226: UserWarning: WARN: Casting input x to numpy array.
  logger.warn("Casting input x to numpy array.")
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:167: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space with exception: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.
  logger.warn(f"{pre} is not with

   239/80000: episode: 1, duration: 13.749s, episode steps: 239, steps per second:  17, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 0.682 [0.000, 3.000],  loss: 0.001754, mae: 0.193631, mean_q: 0.028738
   606/80000: episode: 2, duration: 15.481s, episode steps: 367, steps per second:  24, episode reward:  3.000, mean reward:  0.008 [ 0.000,  1.000], mean action: 1.499 [0.000, 3.000],  loss: 0.001400, mae: 0.010382, mean_q: 0.015752
  1030/80000: episode: 3, duration: 17.436s, episode steps: 424, steps per second:  24, episode reward:  3.000, mean reward:  0.007 [ 0.000,  1.000], mean action: 1.585 [0.000, 3.000],  loss: 0.002717, mae: 0.032121, mean_q: 0.046273
  1400/80000: episode: 4, duration: 16.888s, episode steps: 370, steps per second:  22, episode reward:  4.000, mean reward:  0.011 [ 0.000,  1.000], mean action: 1.216 [0.000, 3.000],  loss: 0.002737, mae: 0.050990, mean_q: 0.073396
  1660/80000: episode: 5, duration: 10.923s, episode steps: 260,

In [24]:
import gym
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import GreedyQPolicy
from gym.wrappers import AtariPreprocessing

class CompatibleAtariPreprocessing(gym.Wrapper):
    def __init__(self, env, **kwargs):
        super().__init__(env)
        self.env = AtariPreprocessing(env, **kwargs)

    def step(self, action):
        observation, reward, done, info, truncated = self.env.step(action)
        # Ensure info is a dictionary
        if not isinstance(info, dict):
            info = {}  # or handle appropriately if there's meaningful data in 'info'
        return observation, reward, done, info  # return only the first four elements

# Environment setup with custom preprocessing wrapper
env = gym.make('Breakout-v4')
env = CompatibleAtariPreprocessing(env, screen_size=84, grayscale_obs=True, frame_skip=1, scale_obs=True, new_step_api=True)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n


# Adjusting the model to the preprocessed observation space
input_shape = (1,) + env.observation_space.shape
model = Sequential()
model.add(Flatten(input_shape=input_shape))
model.add(Dense(24, activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(nb_actions, activation='linear'))

# Configure the agent
memory = SequentialMemory(limit=50000, window_length=1)
policy = GreedyQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, policy=policy)
dqn.compile(optimizer=Adam())

# Train the agent
dqn.fit(env, nb_steps=50000, visualize=False, verbose=2)

# Save the final policy network after training
dqn.save_weights('policy_final.h5', overwrite=True)



/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Training for 50000 steps ...


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:137: UserWarning: WARN: The obs returned by the `reset()` method was expecting a numpy array, actual type: <class 'tuple'>
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:226: UserWarning: WARN: Casting input x to numpy array.
  logger.warn("Casting input x to numpy array.")
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:167: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space with exception: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.
  logger.warn(f"{pre} is not with

  1424/50000: episode: 1, duration: 21.242s, episode steps: 1424, steps per second:  67, episode reward:  4.000, mean reward:  0.003 [ 0.000,  1.000], mean action: 1.575 [0.000, 2.000],  loss: 0.001543, mean_q: 0.068403
  1859/50000: episode: 2, duration: 16.622s, episode steps: 435, steps per second:  26, episode reward:  4.000, mean reward:  0.009 [ 0.000,  1.000], mean action: 0.841 [0.000, 2.000],  loss: 0.002399, mean_q: 0.006847
  2142/50000: episode: 3, duration: 11.404s, episode steps: 283, steps per second:  25, episode reward:  2.000, mean reward:  0.007 [ 0.000,  1.000], mean action: 0.721 [0.000, 2.000],  loss: 0.002489, mean_q: -0.002551
  2571/50000: episode: 4, duration: 16.421s, episode steps: 429, steps per second:  26, episode reward:  2.000, mean reward:  0.005 [ 0.000,  1.000], mean action: 0.303 [0.000, 3.000],  loss: 0.002557, mean_q: -0.000348
  2813/50000: episode: 5, duration: 10.061s, episode steps: 242, steps per second:  24, episode reward:  0.000, mean rewa